<a href="https://colab.research.google.com/github/jos0213/kaggle/blob/main/public_0_86457_LightGBM_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 데이터 처리 및 분석
import pandas as pd
import numpy as np

# 모델 학습 및 평가
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# 데이터 시각화
import seaborn as sns
import matplotlib.pyplot as plt

# 데이터 불균형 문제 해결
from imblearn.over_sampling import SMOTE

# 머신러닝 모델
import lightgbm as lgb

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
# 데이터 로드
train = pd.read_csv('/content/drive/MyDrive/DACON/2025/(202502) 부동산 허위매물 분류 해커톤: 가짜를 색출하라!/open.zip (Unzipped Files)/train.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON/2025/(202502) 부동산 허위매물 분류 해커톤: 가짜를 색출하라!/open.zip (Unzipped Files)/test.csv')

In [4]:
# === 결측치 처리 ===
columns_fill_knn = ['해당층', '총층', '전용면적', '방수', '욕실수', '총주차대수']
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
train[columns_fill_knn] = imputer.fit_transform(train[columns_fill_knn])
test[columns_fill_knn] = imputer.transform(test[columns_fill_knn])

In [5]:
# === 레이블 인코딩 ===
from sklearn.preprocessing import LabelEncoder
label_encode_cols = ['중개사무소', '게재일', '제공플랫폼', '방향']
for col in label_encode_cols:
    le = LabelEncoder()
    combined_data = pd.concat([train[col], test[col]], axis=0).astype(str)
    le.fit(combined_data)
    train[col] = le.transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))

In [6]:
# === 원-핫 인코딩 ===
from sklearn.preprocessing import OneHotEncoder
one_hot_cols = ['매물확인방식', '주차가능여부']
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
train_encoded = one_hot_encoder.fit_transform(train[one_hot_cols])
test_encoded = one_hot_encoder.transform(test[one_hot_cols])
train = pd.concat([train.drop(columns=one_hot_cols), pd.DataFrame(train_encoded, index=train.index)], axis=1)
test = pd.concat([test.drop(columns=one_hot_cols), pd.DataFrame(test_encoded, index=test.index)], axis=1)

In [7]:
# === ID 컬럼 제거 ===
train = train.drop(columns=['ID'])
test_id = test['ID']
test = test.drop(columns=['ID'])

In [8]:
# 데이터 분리
X = train.drop(columns=['허위매물여부',0])
y = train['허위매물여부']

# 컬럼 이름을 문자열로 변환
X.columns = X.columns.astype(str)

# SMOTE로 데이터 증강
smote = SMOTE(random_state=42)
X_sm, y_sm = smote.fit_resample(X, y)

# 확인
print("After SMOTE, X_sm shape:", X_sm.shape)
print("After SMOTE, y_sm distribution:\n", pd.Series(y_sm).value_counts())


After SMOTE, X_sm shape: (4308, 13)
After SMOTE, y_sm distribution:
 허위매물여부
0    2154
1    2154
Name: count, dtype: int64


In [9]:
# === 사용자 정의 F1 평가 지표 ===
def f1_metric(y_pred, data):
    y_true = data.get_label()
    y_pred_binary = (y_pred > 0.5).astype(int)
    f1 = f1_score(y_true, y_pred_binary)
    return 'f1', f1, True

In [10]:
# === LightGBM 모델 학습 ===
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'class_weight': 'balanced',
    'seed': 42
}

In [11]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.6 MB/s eta 0:00:00


In [13]:
import optuna
from sklearn.metrics import f1_score

def objective(trial):
    # 하이퍼파라미터 튜닝
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 16, 256),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 0.9),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.7, 0.9),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'seed': 42,
        'class_weight': 'balanced',
        'metric': 'f1',
        'verbose': -1
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    oof_preds = np.zeros(len(X_sm))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_sm, y_sm)):
        X_train, X_val = X_sm.iloc[train_idx], X_sm.iloc[val_idx]
        y_train, y_val = y_sm.iloc[train_idx], y_sm.iloc[val_idx]

        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val)

        # 콜백으로 early stopping 설정
        callbacks = [
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(0)  # 로그 출력을 최소화
        ]

        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            feval=f1_metric,  # 사용자 정의 F1 평가 함수
            num_boost_round=500,
            callbacks=callbacks
        )

        # 예측 저장
        oof_preds[val_idx] = model.predict(X_val)

    f1 = f1_score(y_sm, (oof_preds > 0.5).astype(int))
    return f1

# Optuna 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# 최적 하이퍼파라미터 출력
print("Best parameters:", study.best_params)

[I 2025-02-09 14:06:44,413] A new study created in memory with name: no-name-0b9fd423-2ae6-4e64-854d-ed33a6a7360a
<ipython-input-13-24a02e8ae29f>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
<ipython-input-13-24a02e8ae29f>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 0.9),
<ipython-input-13-24a02e8ae29f>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('baggin

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[16]	valid_0's f1: 0.927336
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[416]	valid_0's f1: 0.973654
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[278]	valid_0's f1: 0.961276
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[88]	valid_0's f1: 0.951724
Training until validation scores don't improve for 50 rounds


[I 2025-02-09 14:06:56,225] Trial 0 finished with value: 0.955285484980509 and parameters: {'num_leaves': 29, 'learning_rate': 0.016165118396018432, 'max_depth': 9, 'feature_fraction': 0.8949608027899466, 'bagging_fraction': 0.8329742890778338, 'lambda_l1': 2.07891742670705, 'lambda_l2': 0.01609454749456613, 'min_data_in_leaf': 18}. Best is trial 0 with value: 0.955285484980509.


Early stopping, best iteration is:
[292]	valid_0's f1: 0.962199
Training until validation scores don't improve for 50 rounds


<ipython-input-13-24a02e8ae29f>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
<ipython-input-13-24a02e8ae29f>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 0.9),
<ipython-input-13-24a02e8ae29f>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.7, 0.9),
<ipython-input-13-24a02e8ae29f>:14: FutureWarning: suggest_loguniform has been deprecated 

...

Early stopping, best iteration is:
[106]	valid_0's f1: 0.978085
Training until validation scores don't improve for 50 rounds


<ipython-input-13-24a02e8ae29f>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
<ipython-input-13-24a02e8ae29f>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 0.9),
<ipython-input-13-24a02e8ae29f>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.7, 0.9),
<ipython-input-13-24a02e8ae29f>:14: FutureWarning: suggest_loguniform has been deprecated 

Early stopping, best iteration is:
[118]	valid_0's f1: 0.957714
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[238]	valid_0's f1: 0.979405
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[127]	valid_0's f1: 0.96789
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[145]	valid_0's f1: 0.973472
Training until validation scores don't improve for 50 rounds


[I 2025-02-09 14:11:17,364] Trial 49 finished with value: 0.970615243342516 and parameters: {'num_leaves': 214, 'learning_rate': 0.0999934371249228, 'max_depth': 6, 'feature_fraction': 0.6050704336409714, 'bagging_fraction': 0.8371854244744164, 'lambda_l1': 0.030979132470983477, 'lambda_l2': 9.913078255583502, 'min_data_in_leaf': 30}. Best is trial 32 with value: 0.9742410303587856.


Early stopping, best iteration is:
[135]	valid_0's f1: 0.974654
Best parameters: {'num_leaves': 171, 'learning_rate': 0.08056540387365017, 'max_depth': 7, 'feature_fraction': 0.6234234753246524, 'bagging_fraction': 0.8350901769307447, 'lambda_l1': 0.15587808288370886, 'lambda_l2': 0.0010000913881142053, 'min_data_in_leaf': 38}


In [15]:
# optuna로 찾은 best params 저장
best_params = study.best_params
best_params.update({
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'f1',
    'class_weight': 'balanced',
    'seed': 42
})

In [19]:
# oof_preds와 test_preds 초기화 (SMOTE 이후 데이터 크기로 조정)
test_preds = np.zeros(len(test))

# 최종 모델 학습
train_data = lgb.Dataset(X_sm, label=y_sm)
final_model = lgb.train(
    best_params,
    train_data,
    num_boost_round=500
)

# 예측 및 저장
test_preds += final_model.predict(test)

In [20]:
# === 테스트 데이터 예측 및 저장 ===
test_preds_binary = (test_preds > 0.5).astype(int)
submission = pd.DataFrame({'ID': test_id, '허위매물여부': test_preds_binary})
submission.to_csv('/content/drive/MyDrive/DACON/2025/(202502) 부동산 허위매물 분류 해커톤: 가짜를 색출하라!/submission_lgbm_optuna.csv', index=False)
print("Submission file saved to 'submission_lgbm_optuna.csv'")

Submission file saved to 'submission_lgbm_optuna.csv'
